In [0]:
# Upgrade Databricks SDK to the latest version and restart Python to see updated packages
%pip install --upgrade databricks-sdk==0.49.0
%restart_python

In [0]:
from databricks.sdk.service.jobs import JobSettings as Job
from databricks.sdk import WorkspaceClient


In [0]:
w = WorkspaceClient()

jobnintendo = None

for job in w.jobs.list():
    if job.settings.name == "Nintendo Pipeline":
        jobnintendo = job.job_id
    elif job.settings.name == "Nintendo Pipeline":
        jobnintendo = job.job_id

print(f"Job ID Nintendo Pipeline: {jobnintendo}")


In [0]:
w = WorkspaceClient()

dltpipeline = None

# Lista todos os pipelines
for p in w.pipelines.list_pipelines():
    if p.name == "DLT - nintendo":
        dltpipeline = p.pipeline_id

print(f"Job ID DLT - nintendo: {dltpipeline}")

In [0]:
dlt_pipeline = Job.from_dict(
    {
        "id": "6f61be34-7a80-430b-b00b-f6709fab3e7f",
        "pipeline_type": "WORKSPACE",
        "clusters": [
            {
                "label": "default",
                "node_type_id": "Standard_D4s_v3",
                "custom_tags": {
                    "pipelinetype": "DLT"
                },
                "autoscale": {
                    "min_workers": 1,
                    "max_workers": 2,
                    "mode": "ENHANCED"
                }
            },
            {
                "label": "maintenance",
                "custom_tags": {
                    "pipelinetype": "DLT"
                }
            }
        ],
        "development": False,
        "notifications": [
            {
                "email_recipients": [
                    "felipegoraro@outlook.com.br"
                ],
                "alerts": [
                    "on-update-failure",
                    "on-update-fatal-failure",
                    "on-flow-failure"
                ]
            }
        ],
        "continuous": False,
        "channel": "CURRENT",
        "photon": False,
        "libraries": [
            {
                "notebook": {
                    "path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/dlt-ingest-table"
                }
            },
            {
                "notebook": {
                    "path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/dlt-clean-table"
                }
            },
            {
                "notebook": {
                    "path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/dlt-fine-table"
                }
            },
            {
                "notebook": {
                    "path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/dlt-normalize"
                }
            },
            {
                "notebook": {
                    "path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/dlt-metrics"
                }
            }
        ],
        "name": "DLT - nintendo",
        "edition": "ADVANCED",
        "catalog": "nintendodatabricks_workspace",
        "configuration": {
            "fs.azure.account.key.nintendostorageaccount.dfs.core.windows.net": "{{secrets/nintendo-keyvault-scope/nintendo-storage-account-key-value}}"
        },
        "schema": "gold"
    }
)

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
try:
    w.jobs.reset(new_settings=dlt_pipeline, job_id=dltpipeline)
except:
    w.jobs.create(**dlt_pipeline.as_shallow_dict())


In [0]:
job_pipeline = Job.from_dict(
    {
    "name": "Nintendo Pipeline",
    "description": "Pipeline de dados referente a processos de arquitetura medalion do projeto nintendo",
    "email_notifications": {
        "on_failure": [
        "felipegoraro@outlook.com"
        ],
        "on_duration_warning_threshold_exceeded": [
        "felipegoraro@outlook.com"
        ],
        "no_alert_for_skipped_runs": False
    },
    "webhook_notifications": {},
    "notification_settings": {
        "no_alert_for_skipped_runs": False,
        "no_alert_for_canceled_runs": False
    },
    "timeout_seconds": 2100,
    "health": {
        "rules": [
        {
            "metric": "RUN_DURATION_SECONDS",
            "op": "GREATER_THAN",
            "value": 1800
        }
        ]
    },
    "trigger": {
        "pause_status": "UNPAUSED",
        "file_arrival": {
        "url": "abfss://nintendo@nintendostorageaccount.dfs.core.windows.net/inbound/"
        }
    },
    "max_concurrent_runs": 1,
    "tasks": [
        {
        "task_key": "DLT_Nintendo",
        "run_if": "ALL_SUCCESS",
        "pipeline_task": {
            "pipeline_id": "6f61be34-7a80-430b-b00b-f6709fab3e7f",
            "full_refresh": False
        },
        "timeout_seconds": 0,
        "email_notifications": {},
        "webhook_notifications": {}
        },
        {
        "task_key": "Test-Ingest",
        "depends_on": [
            {
            "task_key": "DLT_Nintendo"
            }
        ],
        "run_if": "ALL_SUCCESS",
        "notebook_task": {
            "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/test/ingest-test",
            "source": "WORKSPACE"
        },
        "job_cluster_key": "job-cluster-nintendo",
        "timeout_seconds": 0,
        "email_notifications": {},
        "webhook_notifications": {}
        },
        {
        "task_key": "Test-Clean",
        "depends_on": [
            {
            "task_key": "Test-Ingest"
            }
        ],
        "run_if": "ALL_SUCCESS",
        "notebook_task": {
            "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/test/clean-test",
            "source": "WORKSPACE"
        },
        "job_cluster_key": "job-cluster-nintendo",
        "timeout_seconds": 0,
        "email_notifications": {},
        "webhook_notifications": {}
        },
        {
        "task_key": "Test-Fine",
        "depends_on": [
            {
            "task_key": "Test-Clean"
            }
        ],
        "run_if": "ALL_SUCCESS",
        "notebook_task": {
            "notebook_path": "/Repos/felipegoraro@outlook.com/projeto_nintendo/src/test/fine-test",
            "source": "WORKSPACE"
        },
        "job_cluster_key": "job-cluster-nintendo",
        "timeout_seconds": 0,
        "email_notifications": {}
        }
    ],
    "job_clusters": [
        {
        "job_cluster_key": "job-cluster-nintendo",
        "new_cluster": {
            "cluster_name": "",
            "spark_version": "15.4.x-scala2.12",
            "spark_conf": {
            "spark.master": "local[*, 4]",
            "spark.databricks.cluster.profile": "singleNode",
            "fs.azure.account.key.nintendostorageaccount.dfs.core.windows.net": "{{secrets/nintendo-keyvault-scope/nintendo-storage-account-key-value}}"
            },
            "azure_attributes": {
            "first_on_demand": 1,
            "availability": "SPOT_WITH_FALLBACK_AZURE",
            "spot_bid_max_price": -1
            },
            "node_type_id": "Standard_D4s_v3",
            "driver_node_type_id": "Standard_D4s_v3",
            "custom_tags": {
            "ResourceClass": "SingleNode"
            },
            "enable_elastic_disk": True,
            "data_security_mode": "SINGLE_USER",
            "runtime_engine": "STANDARD",
            "num_workers": 0
        }
        }
    ],
    "tags": {
        "project": "nintendo",
        "type": "job"
    },
    "queue": {
        "enabled": True
    },
    "performance_target": "PERFORMANCE_OPTIMIZED",
    "run_as": {
        "user_name": "felipegoraro@outlook.com"
    }
    }
)

w = WorkspaceClient()
try:
    w.jobs.reset(new_settings=job_pipeline, job_id=jobnintendo)
except:
    w.jobs.create(**job_pipeline.as_shallow_dict())
